# `System.DateTime` and JavaScript dates

Here I will write down some basic observations about the `DateTime` `struct` in .NET [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.datetime?view=net-7.0)] and the `Date` object in JavaScript [📖 [docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Date)]. These remarks appear here after programming for 30 years so I apologize in advance for not having any depth in this area as much as, say, Jon Skeet.

## .NET: there is no need to call `DateTime.Now.ToUniversalTime()`



There is no need to call `DateTime.Now.ToUniversalTime()` because we have `DateTime.UtcNow`:

In [1]:
DateTime.Now.ToUniversalTime().ToString()

10/22/2023 10:24:53 PM

In [2]:
DateTime.UtcNow.ToString()

10/22/2023 10:24:54 PM

According to the “[Standard date and time format strings](https://learn.microsoft.com/en-us/dotnet/standard/base-types/standard-date-and-time-format-strings)” of .NET the string format of the dates shown above are of a locale-specific [date and time format](https://learn.microsoft.com/en-us/dotnet/standard/base-types/standard-date-and-time-format-strings#date-and-time-formats). Specifically, “[the general date long time ("G") format specifier](https://learn.microsoft.com/en-us/dotnet/standard/base-types/standard-date-and-time-format-strings#the-general-date-long-time-g-format-specifier)” which we can state explicitly:

In [3]:
using System.Globalization;

DateTime.UtcNow.ToString("G", CultureInfo.CreateSpecificCulture("en-us"))

10/22/2023 10:24:54 PM

## JavaScript: the `Date` constructor takes “a simplification of the ISO 8601 calendar date extended format”



We cannot pass the locale-specific date string from .NET above into the `Date` constructor and expect that date (and time) to be _transliterated_ into a JavaScript `Date`. Let us try `3/7/2004 6:09:03 PM`:

In [4]:
#!javascript

const dateFrom2004 = new Date('3/7/2004 6:09:03 PM');

console.log(`Is the year 2004? [${dateFrom2004.getFullYear() === 2004}]`); // `.getYear()` is deprecated!
console.log(`Is the month 3? [${dateFrom2004.getMonth() === 3}, because \`.getMonth()\` is zero-based 😐]`);
console.log(`Is the month-day 7? [${dateFrom2004.getDate() === 7}]`); // `.getDay()` returns the week-day
console.log(`Is the hour 6? [${dateFrom2004.getHours() === 6}: changed to 24hr format 🔥]`);
console.log(`Are the minutes 9? [${dateFrom2004.getMinutes() === 9}]`);
console.log(`\`.toString()\`: ${dateFrom2004.toString()}`);
console.log(`\`.toLocaleString('en-US')\`: ${dateFrom2004.toLocaleString('en-US')}`);
console.log(`\`.toUTCString()\`: ${dateFrom2004.toUTCString()}`);
console.log(`\`.toISOString()\`: ${dateFrom2004.toISOString()}`);

Is the year 2004? [true]

Is the month 3? [false, because `.getMonth()` is zero-based 😐]

Is the month-day 7? [true]

Is the hour 6? [false: changed to 24hr format 🔥]

Are the minutes 9? [true]

`.toString()`: Sun Mar 07 2004 18:09:03 GMT-0800 (Pacific Standard Time)

`.toLocaleString('en-US')`: 3/7/2004, 6:09:03 PM

`.toUTCString()`: Mon, 08 Mar 2004 02:09:03 GMT

`.toISOString()`: 2004-03-08T02:09:03.000Z

Apart from the weirdness of `.getMonth()` [📖 [docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Date/getMonth)] being zero-based, the hour of the original date was changed to the 24-hour format. Such a small change could cause a big bug when these nuances are not expected and/or forgotten about…

According to [the MDN docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Date#date_time_string_format), this is the format template that is the “one format to be universally supported”:

```plaintext
YYYY-MM-DDTHH:mm:ss.sssZ
```

We have seen this format in the JavaScript output above in the rendering of `.toISOString()` [📖 [docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Date/toISOString)]. And it is important to remember that the punctuating Z is “[the zone designator for the zero UTC offset](https://en.wikipedia.org/wiki/ISO_8601).”

## .NET: using the “universally supported” JavaScript format



Does the “universally supported” template, `YYYY-MM-DDTHH:mm:ss.sssZ`, work in .NET verbatim?

In [5]:
var dateStringFrom2004 = "3/7/2004 6:09:03 PM";

DateTime.Parse(dateStringFrom2004).ToString("YYYY-MM-DDTHH:mm:ss.sssZ")

YYYY-03-DDT18:09:03.03Z

The short answer is _no_. After a review of “[Custom date and time format strings](https://learn.microsoft.com/en-us/dotnet/standard/base-types/custom-date-and-time-format-strings),” we may see this:

```plaintext
yyyy'-'MM'-'dd'T'HH':'mm':'ss'.'fff'Z'
```

In [6]:
const string iso8601Template = "yyyy'-'MM'-'dd'T'HH':'mm':'ss'.'fff'Z'";

var isoDateStringFrom2004 = DateTime.Parse(dateStringFrom2004).ToString(iso8601Template);

isoDateStringFrom2004

2004-03-07T18:09:03.000Z

It is important to mention that this .NET template for the [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601) format is _not_ among the “[Standard date and time format strings](https://learn.microsoft.com/en-us/dotnet/standard/base-types/standard-date-and-time-format-strings)” of .NET. And this template can easily be confused with `UniversalSortableDateTimePattern` [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.globalization.datetimeformatinfo.universalsortabledatetimepattern?view=net-7.0)], `yyyy'-'MM'-'dd HH':'mm':'ss'Z'`.

We can now use our ‘ISO date’, `2004-03-07T18:09:03.000Z`, in the JavaScript we had earlier:

In [7]:
#!javascript

#!share --from csharp isoDateStringFrom2004

const dateFrom2004 = new Date(isoDateStringFrom2004);

console.log(`Is the year 2004? [${dateFrom2004.getFullYear() === 2004}]`);
console.log(`Is the month 3? [${dateFrom2004.getMonth() === 3}, because \`.getMonth()\` is zero-based 😐]`);
console.log(`Is the month-day 7? [${dateFrom2004.getDate() === 7}]`);
console.log(`Is the hour 18? [${dateFrom2004.getHours() === 18}], because the locale hour is ${dateFrom2004.getHours()}`);
console.log(`Is the UTC hour 18? [${dateFrom2004.getUTCHours() === 18}]`);
console.log(`Are the minutes 9? [${dateFrom2004.getMinutes() === 9}]`);
console.log(`\`.toString()\`: ${dateFrom2004.toString()}`);
console.log(`\`.toLocaleString('en-US')\`: ${dateFrom2004.toLocaleString('en-US')}`);
console.log(`\`.toUTCString()\`: ${dateFrom2004.toUTCString()}`);
console.log(`\`.toISOString()\`: ${dateFrom2004.toISOString()}`);

Is the year 2004? [true]

Is the month 3? [false, because `.getMonth()` is zero-based 😐]

Is the month-day 7? [true]

Is the hour 18? [false], because the locale hour is 10

Is the UTC hour 18? [true]

Are the minutes 9? [true]

`.toString()`: Sun Mar 07 2004 10:09:03 GMT-0800 (Pacific Standard Time)

`.toLocaleString('en-US')`: 3/7/2004, 10:09:03 AM

`.toUTCString()`: Sun, 07 Mar 2004 18:09:03 GMT

`.toISOString()`: 2004-03-07T18:09:03.000Z

Note how we _still_ failed to transliterate the hour, according to `.getHours()`. We had to resort to `.getUTCHours()` [📖 [docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Date/getUTCHours)] to find the expected UTC hour. What JavaScript is telling us is:

- When a UTC ISO string is constructed by `Date`, the UTC information will be translated into the locale.
- The original UTC information is available via several `.getUTC*` getters.

The StackOverflow question, “[How to convert a Date to UTC?](https://stackoverflow.com/questions/948532/how-to-convert-a-date-to-utc),” asked over 14 years ago, looks like a misleading question!

Is this locale-biased design also in .NET?

In [8]:
var dateFrom2004 = DateTime.Parse(isoDateStringFrom2004);

@$"
Input: {isoDateStringFrom2004}

Is the year 2004? [{dateFrom2004.Year == 2004}]
Is the month 3? [{dateFrom2004.Month == 3}]
Is the month-day 7? [{dateFrom2004.Day == 7}]
Is the hour 18? [{dateFrom2004.Hour == 18}, because the locale hour is {dateFrom2004.Hour}]
Is the UTC hour 18? [{dateFrom2004.ToUniversalTime().Hour == 18}]
Are the minutes 9? [{dateFrom2004.Minute == 9}]
JavaScript locale: {dateFrom2004:M/d/yyyy, HH':'mm':'ss tt}
ISO: {dateFrom2004:yyyy'-'MM'-'dd'T'HH':'mm':'ss'.'fff'Z'}
"


Input: 2004-03-07T18:09:03.000Z

Is the year 2004? [True]
Is the month 3? [True]
Is the month-day 7? [True]
Is the hour 18? [False, because the locale hour is 10]
Is the UTC hour 18? [True]
Are the minutes 9? [True]
JavaScript locale: 3/7/2004, 10:09:03 AM
ISO: 2004-03-07T10:09:03.000Z


The short answer is _yes_. It takes a call to `.ToUniversalTime()` [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.datetime.touniversaltime?view=net-7.0)] to obtain the UTC information.

By default, I am not seeing how we can obtain JavaScript phrases like `(Pacific Standard Time)` or `GMT` so I have avoided replicating `.toString()`, `.toUTCString()` and `.toISOString()` output in .NET.

## JavaScript: `Date.parse()` does _not_ return a `Date` object



The [MDN docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Date/parse) for `Date.parse()` open with this:

>The `Date.parse()` static method parses a string representation of a date, and returns the date’s timestamp.

We can take our ‘ISO string,’ `isoDateStringFrom2004`, from above to verify this:

In [9]:
#!javascript

#!share --from csharp isoDateStringFrom2004
const timestamp = Date.parse(isoDateStringFrom2004);

console.log(`Does \`Date.parse\` return a date? [${timestamp instanceof Date}]`);
console.log(`Does \`Date.parse\` return a number? [${typeof timestamp === 'number'}]`);
console.log(`\`Date.parse\` returns ${timestamp}.`);

Does `Date.parse` return a date? [false]

Does `Date.parse` return a number? [true]

`Date.parse` returns 1078682943000.

## JavaScript: the `Date.UTC()` static method does not take timestamp numbers



The [MDN docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Date/UTC#behavior_of_date.utc_with_one_argument) get to the point:

>`Date.UTC()` when passed one argument used to have inconsistent behavior, because implementations only kept the behavior consistent with the `Date()` constructor, which does not interpret a single argument as the year number.

Being “consistent with the `Date()` constructor” means changing the meaning of the first argument from `year` to `timestamp`. This anomalous detail is now gone from JavaScript.

In [10]:
#!javascript

#!share --from csharp isoDateStringFrom2004
const timestamp = Date.parse(isoDateStringFrom2004);

console.log(`\`Date.parse\` returns ${timestamp}.`);

const timestampDate = Date.UTC(timestamp);
console.log(`timestampDate: ${timestampDate}`);

`Date.parse` returns 1078682943000.

timestampDate: NaN

## JavaScript: the ISO 8601 string format is designed to be sortable

Let us use the power of .NET to generate some ‘ISO strings’ from `isoDateStringFrom2004`:

In [11]:
var dateFrom2004 = DateTime.Parse(isoDateStringFrom2004);

@$"
{isoDateStringFrom2004},
{dateFrom2004.AddDays(-2).AddMinutes(5).ToUniversalTime().ToString(iso8601Template)},
{dateFrom2004.AddMinutes(-20).AddSeconds(-13).ToUniversalTime().ToString(iso8601Template)},
{dateFrom2004.AddDays(-1).AddHours(-6).AddSeconds(-33).ToUniversalTime().ToString(iso8601Template)},
"


2004-03-07T18:09:03.000Z,
2004-03-05T18:14:03.000Z,
2004-03-07T17:48:50.000Z,
2004-03-06T12:08:30.000Z,


We can pass these strings to JavaScript and sort:

In [12]:
#!javascript

const strings = `
    2004-03-07T18:09:03.000Z,
    2004-03-05T18:14:03.000Z,
    2004-03-07T17:48:50.000Z,
    2004-03-06T12:08:30.000Z
`;

const data = strings.split(',').map(s => s.trim()).sort();

data.forEach((s, i) => {
    console.log((i === data.length - 1) ? `max: ${s}` : s);
});


2004-03-05T18:14:03.000Z

2004-03-06T12:08:30.000Z

2004-03-07T17:48:50.000Z

max: 2004-03-07T18:09:03.000Z

The StackOverflow question, “[Min/Max of dates in an array?](https://stackoverflow.com/questions/7143399/min-max-of-dates-in-an-array?rq=3),” asked over 12 years ago, does not offer the sortability of date strings as a solution.

## <!-- -->

🐙🐱[BryanWilhite](https://github.com/BryanWilhite)